# Import

In [3]:
from investment.utils import get_date_parameter, budget_item_transaction_monthly_recon, update_foreign_key
from investment.models import Budget, FinancialManager, SpendingTheme,\
    FinancialInstitutions, Transaction, CashFlowAccountingCategorie, Merchant, TransactionSource,\
    UploadedPDF


In [1]:
def budget_item_transaction_monthly_recon(request, budget_item, date_dict):
    # Combine queries for last month and current month
    from .models import 
    # this takes into considerentaion all transaction sonce inception
    all_transactions_as_of_last_month = Transaction.objects.filter(
        user__id=request.user.id,
        category__id=budget_item.id,
        date__lte=date_dict['previous_month_eom_date']
    ).order_by('date')

    all_transactions_as_of_this_begining_of_year = Transaction.objects.filter(
        user__id=request.user.id,
        category__id=budget_item.id,
        date__lt=date_dict['boy']
    ).order_by('date')

    this_month_transaction = Transaction.objects.filter(
        user__id=request.user.id,
        category__id=budget_item.id,
        date__range=(date_dict['b'], date_dict['e'])
    ).order_by('date')

    # Check for next and previous month existence
    next_month_transactions_exists = date_dict["evaluation_date"] < timezone.make_aware(datetime.today())
    previous_month_transactions_exists = date_dict["evaluation_date"] > timezone.make_aware(datetime(date_dict["evaluation_date"].year, 1, 1))

    # Calculate total_spent and current_month_transactions_total directly in the database
    total_spent_last_year = all_transactions_as_of_this_begining_of_year.aggregate(total_spent=Sum('amount'))['total_spent'] or 0
    total_spent = all_transactions_as_of_last_month.aggregate(total_spent=Sum('amount'))['total_spent'] or 0
    current_month_transactions_total = this_month_transaction.aggregate(current_month_total=Sum('amount'))['current_month_total'] or 0

    # Calculate total_budgeted once and reuse
    total_budgeted = budget_item.calculate_amount() * date_dict["previous_month_eom_date"].month if date_dict["previous_month_eom_date"].month != 12 else 0

    # Calculate carried_over_amount without rounding first
    carried_over_amount = total_budgeted - (total_spent - total_spent_last_year)

    context = {
        'transaction_total': round(current_month_transactions_total, 2),
        'transactions': this_month_transaction,
        'budget_item': budget_item,
        "previous_month_transfer": round(carried_over_amount, 2),
        'previous_month_exists': previous_month_transactions_exists,
        'next_month_exists': next_month_transactions_exists,
        'previous_month_url': reverse('transactions-by-budget-titem-view', kwargs={'budget_item_id': budget_item.id}),
        'next_month_url': reverse('transactions-by-budget-titem-view', kwargs={'budget_item_id': budget_item.id}),
        "difference": round(budget_item.calculate_amount() + carried_over_amount - current_month_transactions_total, 2)
    }

    return context
